# Lesson 3 Exercise: Read Text With Computer Vision Service

Add a brief description of the work being done in this notebook...

## Setup the notebook

To access the Custom Vision service from this Python notebook, you need to install the Azure Cognitive Services Custom Vision Library. This library is part of the [Azure SDK for Python](https://github.com/Azure/azure-sdk-for-python) GitHub project.

> To learn more, read the [Azure Cognitive Services modules for Python](https://docs.microsoft.com/python/api/overview/azure/cognitive-services?view=azure-python) article in Microsoft Docs.

Execute the cell below to install the library.

In [ ]:
pip install azure-cognitiveservices-vision-computervision

Run the following cell to import the libraries and services required to execute the cells below.

In [ ]:
# Import the libraries need to access the Computer Vision services
from azure.cognitiveservices.vision.computervision import ComputerVisionClient
from azure.cognitiveservices.vision.computervision.models import OperationStatusCodes
from msrest.authentication import CognitiveServicesCredentials

# Import a few utility libraries
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw
import os
import time

## Set variables

To access your Computer Vision service, its authentication key and endpoint URL need to be supplied to client applications. Using the values from the `Keys and Endpoints` page for you Computer Vision service, replace the tokenized values in the cell below, as follows:

- Retrieve the `Key 1` value for your Computer Vision service in the Azure portal and update the `key` value below.
- Retrieve the `Endpoint` value for your Computer Vision in the Azure portal and update the `endpoint` value below.

Execute the cell below to set the variables for this notebook.

In [ ]:
key = 'YOUR_COGNITIVE_SERVICES_KEY'
endpoint = 'YOUR_COGNITIVE_SERVICES_ENDPOINT'

print('Ready to perform OCR using the Computer Vision service at "{}" using the key "{}."''"'.format(endpoint, key))

## Create a Computer Vision client

After setting the `key` and `endpoint` needed to access your Computer Vision service, you can instantiate a client.

Execute the cell below to create a `ComputerVisionClient` object.

In [ ]:
# Instantiate a Computer Vision client
client = ComputerVisionClient(endpoint, CognitiveServicesCredentials(key))

## Read text from an image with the OCR API

Use the optical character recognition (OCR) API to extract text from images. The image is of a street sign.

In [ ]:
%matplotlib inline

# Create a matplotlib figure to display the classification results
fig = plt.figure(figsize=(7, 7))

# Read an image file into a stream
path = os.path.join('ocr-images', 'ocr-001.jpg')
stream = open(path, "rb")

# Extract text from the image using the Computer Vision service
ocr = client.recognize_printed_text_in_stream(stream)

text = ''

# Process the OCR text one line at a time
for region in ocr.regions:
    for line in region.lines:
        # Read the OCR'ed text from each line
        for word in line.words:
            text += word.text + ' '
        #print(text.rstrip())

# Display the image with its extracted text
img = Image.open(path)
draw = ImageDraw.Draw(img)
plt.axis('off')
plt.imshow(img)
fig.suptitle(text) 

## Add bounding boxes

Using the OCR API, you can also draw bounding boxes around each unique string of text identified during the OCR process. 

In [ ]:
# Create a matplotlib figure to display the classification results
fig = plt.figure(figsize=(7, 7))

# Display the image with its extracted text inside bounding boxes
img = Image.open(path)
draw = ImageDraw.Draw(img)

# Process the text one line at a time, drawing the appropriate bounding box
for region in ocr.regions:
    for line in region.lines:
        # Draw the bounding box for each line of text
        l,t,w,h = list(map(int, line.bounding_box.split(',')))
        draw.rectangle(((l,t), (l+w, t+h)), outline='magenta', width=5)

# Show the image with the text locations highlighted by bounding boxes
plt.axis('off')
plt.imshow(img)
fig.suptitle(text) 

## Read scanned documents with the Read API

The Computer Vision service include the `Read API`, which can be used to read larger amounts of text from scanned documents, for example.

To use the `Read API`, you must send an image to the Computer Vision service. This will be read and analyzed asynchronously by the service. This means you must await the results and retrieve them when processing is completed.

Execute the cell below to send an image to your Computer Vision service and then wait for the results. When results are available, retrieve and display them.

In [ ]:
%matplotlib inline

# Read an image file into a stream
path = os.path.join('ocr-images', 'covid-diagnosis.jpg')
stream = open(path, "rb")

# Send an async request to read text within the image
operation = client.read_in_stream(stream, raw=True)
# Extract the operation ID from the response headers
locationHeader = operation.headers["Operation-Location"]
operationId = locationHeader.split("/")[-1]

# Wait for the asynchronous operation to complete
while True:
    result = client.get_read_result(operationId)
    if result.status not in [OperationStatusCodes.running]:
        break
    time.sleep(1)

# When the operation has completed successfuly, print each line of text returned to the output
if result.status == OperationStatusCodes.succeeded:
    for res in result.analyze_result.read_results:
        for line in res.lines:
            print(line.text)

# Display the image analyzed for comparision to the OCR'ed text results
print('\n')
fig = plt.figure(figsize=(12,12))
img = Image.open(path)
plt.axis('off')
plt.imshow(img)

## Read Handwritten Text

In addition to printed text, the `Read API` is also capable of reading handwritten text.

Execute the cell below to extract text from a handwritten note.

In [ ]:
%matplotlib inline

# Read an image file into a stream
path = os.path.join('ocr-images', 'handwriting.jpeg')
stream = open(path, "rb")

# Send an async request to read text within the image
operation = client.read_in_stream(stream, raw=True)
# Extract the operation ID from the response headers
locationHeader = operation.headers["Operation-Location"]
operationId = locationHeader.split("/")[-1]

# Wait for the asynchronous operation to complete
while True:
    result = client.get_read_result(operationId)
    if result.status not in [OperationStatusCodes.running]:
        break
    time.sleep(1)

# When the operation has completed successfuly, print each line of text returned to the output
if result.status == OperationStatusCodes.succeeded:
    for res in result.analyze_result.read_results:
        for line in res.lines:
            print(line.text)

# Display the image analyzed for comparision to the OCR'ed text results
print('\n')
fig = plt.figure(figsize=(12,12))
img = Image.open(path)
plt.axis('off')
plt.imshow(img)